<a href="https://colab.research.google.com/github/hoky1227/Transformer_based-recommendation/blob/main/Instacart_Transformer_Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
import csv
import os
import matplotlib.pyplot as plt
import pickle

In [ ]:
path = '/content/drive/MyDrive/Instacart/'
print(os.listdir(path))

def csvread(filename):
    return pd.read_csv(path + filename + '.csv')

# aisles = csvread('aisles')
# departments = csvread('departments')
products = csvread('products')
order_products_train = csvread('order_products__train')
orders = csvread('orders')
order_products_prior = csvread('order_products__prior')
# sample_submission = csvread('sample_submission')

['aisles.csv', 'departments.csv', 'products.csv', 'order_products__train.csv', 'orders.csv', 'order_products__prior.csv', 'sample_submission.csv', 'InstacartDetail.csv']


In [ ]:
def sortdf(data, column):
    data.sort_values(column, inplace=True)
    data.reset_index(inplace=True)
    data.pop('index')
    return data

In [ ]:
products

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


In [ ]:
detail = products['product_name']
detail.to_csv(path + 'InstacartDetail.csv', mode='w')

In [ ]:
products_dic = {products['product_id'][i]: products['product_name'][i] for i in range(len(products))}
products_dic.get(1)

'Chocolate Sandwich Cookies'

In [ ]:
sortdf(orders, ['order_id'])

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,112108,train,4,4,10,9.0
1,2,202279,prior,3,5,9,8.0
2,3,205970,prior,16,5,17,12.0
3,4,178520,prior,36,1,9,7.0
4,5,156122,prior,42,6,16,9.0
...,...,...,...,...,...,...,...
3421078,3421079,108687,prior,6,1,13,0.0
3421079,3421080,52726,prior,2,1,11,2.0
3421080,3421081,117076,prior,1,3,11,NaN
3421081,3421082,175185,prior,23,2,18,4.0


In [ ]:
orders_dic = {orders['order_id'][i]: orders['user_id'][i] for i in range(len(orders))}
orders_dic.get(1)

112108

In [ ]:
order_products = pd.concat([order_products_train, order_products_prior])
order_products.drop(['add_to_cart_order', 'reordered'], axis=1, inplace=True)
# order_products.reset_index(inplace=True)
# order_products.pop('index')
sortdf(order_products, ['order_id'])

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246
3,1,49683
4,1,43633
...,...,...
33819101,3421083,21162
33819102,3421083,45309
33819103,3421083,7854
33819104,3421083,18176


In [ ]:
order_products['user_id'] = order_products['order_id'].apply(lambda x: orders_dic.get(x))
order_products

,order_id,product_id,user_id
0,1,49302,112108
1,1,11109,112108
2,1,10246,112108
3,1,49683,112108
4,1,43633,112108
...,...,...,...
33819101,3421083,21162,25247
33819102,3421083,45309,25247
33819103,3421083,7854,25247
33819104,3421083,18176,25247


In [ ]:
order_products['product_id'] = order_products['product_id'].apply(lambda x: products_dic.get(x))
order_products

,order_id,product_id,user_id
0,1,Bulgarian Yogurt,112108
1,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,112108
2,1,Organic Celery Hearts,112108
3,1,Cucumber Kirby,112108
4,1,Lightly Smoked Sardines in Olive Oil,112108
...,...,...,...
33819101,3421083,Organic Mixed Berry Yogurt & Fruit Snack,25247
33819102,3421083,Purple Carrot & blueberry Puffs,25247
33819103,3421083,Freeze Dried Mango Slices,25247
33819104,3421083,Organic Strawberry Yogurt & Fruit Snack,25247


In [ ]:
sortdf(order_products, ['user_id', 'order_id'])

,order_id,product_id,user_id
0,431534,Organic Fuji Apples,1
1,431534,Bag of Organic Bananas,1
2,431534,Honeycrisp Apples,1
3,431534,Organic String Cheese,1
4,431534,Bartlett Pears,1
...,...,...,...
33819101,3154581,Diet Pepsi Pack,206209
33819102,3154581,Organic Diced Tomatoes,206209
33819103,3154581,Fat Free Cream Cheese,206209
33819104,3186442,Raisin Bran Cereal,206209


In [ ]:
order_products.to_pickle(path + 'order_products.pkl')